In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning/log_files_0712 \
/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning/models_0712 \
/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning



> Get the models at the right place : 

model     LR     weight decay
model     0.0001     0.000001
model_01  1e-6       0.001

In [2]:
import os 

path_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_2411_finetuned"
path_new_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_0712_finetuned"
path_base = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_2809"

path_finetuned_basic = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning/fine_tuning/models_01"
path_finetuned_tailored = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning/fine_tuning/models_0712"

# Move the desired epoch to the ensemble :
tailored_KLtypes = ["KL3" , "KL8", "KL12", "KL13", "KL23", "KL38", "KL46" ,"KL62"]
dico_tailored = {"KL3" : "small", "KL62" : "small"}

epoch = "8"
# Move the finetuned_basic :
for model in os.listdir(path_finetuned_basic) : 
    if model.split(".")[1] == epoch :
        kltype = model.split(".")[0]
        if kltype not in tailored_KLtypes :
            os.system(f"cp {path_finetuned_basic}/{model} {path_new_ensemble}")

# Move the finetuned_tailored :
for model in os.listdir(path_finetuned_tailored) : 
    if model.split(".")[1] == epoch :
        kltype = model.split(".")[0]
        if kltype in tailored_KLtypes :
            os.system(f"cp {path_finetuned_tailored}/{model} {path_new_ensemble}")




In [3]:
# Move the rest :
finetuned_kltypes = [model.split(".")[0] for model in os.listdir(path_new_ensemble)]
for model in os.listdir(path_base) : 
    if model.split(".")[0] not in finetuned_kltypes : 
        os.system(f"cp {path_base}/{model} {path_new_ensemble}")

> Clean the rep

In [1]:
import os 

path_new_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_0712_finetuned"

try :
    os.system(f"rm -rf {path_new_ensemble}")
    os.mkdir(f"{path_new_ensemble}")
except Exception as e :
    print(e)

***

In [4]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models
import TropiGAT_functions

warnings.filterwarnings("ignore")

In [5]:
path_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_0712_finetuned"

dico_models = TropiGAT_functions.make_ensemble_tailored_TropiGAT(path_ensemble , dico_tailored)

> Others 

In [6]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

In [7]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1
 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1


In [8]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

{'AB716666.1_prot_BAP15736.1_24': {'KL110': 0.8094,
                                   'KL123': 0.9985,
                                   'KL125': 0.8789,
                                   'KL127': 0.5852,
                                   'KL13': 0.9444,
                                   'KL14': 0.582,
                                   'KL145': 0.8723,
                                   'KL18': 0.5405,
                                   'KL23': 0.994,
                                   'KL38': 0.7155,
                                   'KL43': 0.5627,
                                   'KL46': 0.716,
                                   'KL5': 0.6357,
                                   'KL51': 0.7684,
                                   'KL57': 0.975,
                                   'KL62': 0.764,
                                   'KL64': 0.547,
                                   'KL7': 0.9995,
                                   'KL70': 0.8128,
                                  

In [9]:
predictions = [other_predictions]

with open(f"/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.finetuned_lvl_{epoch}.0712.results.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

In [10]:
path_results = f"/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.finetuned_lvl_{epoch}.0712.results.tsv"
results_df = pd.read_csv(path_results, sep = "\t", names = ["protein","predictions"])
df_sorted = results_df.sort_values(by='protein', ascending=True)

df_sorted.to_csv(f"/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.finetuned_lvl_{epoch}.ordered.0712.results.tsv", sep = "\t", header = 0, index = False)

